In [ ]:
import requests
import json
from lxml import etree
import time
from urllib import request

# 爬取九个职位
type = ['运营','ai产品','推荐产品','产品经理','java','管培生','数据','python','python爬虫']
#type = ['推荐产品']
for px in type:
    # 每个职位爬27页
    for page in range(2,51):
        
        headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Upgrade-Insecure-Requests': '1',
        'Referer':'https://www.lagou.com/jobs/list_%E8%BF%90%E8%90%A5?labelWords=sug&fromSearch=true&suginput=yu',
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
        }
        
        #设置一个会话
        session = requests.session()

        # 发送Get请求更新cookie
        session.get('https://www.lagou.com/jobs/list_%E8%BF%90%E8%90%A5?labelWords=sug&fromSearch=true&suginput=yu',headers=headers)

        data = {
        'first':'false',
        'pn':page,
        'kd':px
        }

        #使用更新后的hsession请求Ajax json
        rep = session.post('https://www.lagou.com/jobs/positionAjax.json?city=%E5%B9%BF%E5%B7%9E&needAddtionalResult=false',headers=headers,data=data)

        # 请求成功，接下来就是提取想要的信息。
        data = json.loads(rep.text)
        
        # 标记变量
        num = 0
        for row in data['content']['positionResult']['result']:
            try:
                # 存在row['district']为None报错情况
                if row['district']!= None:
                    # 标记次数
                    num +=1
                    url = 'https://www.lagou.com/jobs/%d.html'%row['positionId']
                    print(url)
                    header = {
                    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                    'Accept-Encoding': 'gzip, deflate, br',
                    'Accept-Language': 'zh-CN,zh;q=0.9',
                    'Upgrade-Insecure-Requests': '1',
                    'Referer':url,
                    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
                    }
                    rep = requests.get(url,proxies=get_proxies(),headers=header)

                    html = etree.HTML(rep.text)

                    # 以下为目标信息获取
                    info = ''
                    info += '拉勾'+'~'
                    info += px+'~'
                    info += 'https://www.lagou.com/jobs/list_%s?labelWords=&fromSearch=true&suginput='%px+'~'
                    info += str(row['positionId'])+'~'
                    info += url+'~'
                     
                    info_company = str(html.xpath("//div[@class='job_company_content']//em/text()")).replace(' ','').replace("n",'').replace("\\",'')+"~"
                    info = info + info_company
                    info += str(html.xpath("//ul[@class='c_feature']/li[1]//h4[1]/text()"))+'~'
                    info += str(html.xpath("//ul[@class='c_feature']/li[2]//h4[1]/text()"))+'~'
                    info += str(html.xpath("//ul[@class='c_feature']/li[3]//h4[1]/text()"))+'~'
                    info += str(html.xpath("//ul[@class='c_feature']/li[4]//h4[1]/text()"))+'~'
                    info += str(html.xpath("//ul[@class='c_feature']/li[5]//h4[1]/text()"))+'~'

                    info += str(html.xpath("//h1[@class='name']/text()"))+'~'
                    info += str(html.xpath("//div[@class='job-detail']//p/text()"))+'~'
                    info += str(html.xpath("//span[@class='salary']/text()"))+'~'
                    info += str(html.xpath("//dd[@class='job_request']/h3[1]/span[3]/text()"))+'~'
                    info += str(html.xpath("//dd[@class='job_request']/h3[1]/span[4]/text()"))+'~'
                    info += str(html.xpath("//dd[@class='job_request']/h3[1]/span[2]/text()"))+'~'

                    info += str(html.xpath("//div[@class='work_addr']/a[2]/text()"))+'~'
                    info += str(html.xpath("//div[@class='work_addr']/text()")).replace(' ','').replace("n",'').replace("\\",'').replace("'",'').replace("-",'').replace(",",'')+'~'
                    info += str(html.xpath("//dd[@class='job_request']/h3[1]/span[5]/text()"))+'~'
                    info += str(html.xpath("//p[@class='publish_time']/text()"))
                    info += '\n'
                    
                    if len(info)>200:
                        print(info)
                        with open('job7.xlsx','a') as o:
                            o.write(info)
                    else:
                        continue
                else:
                    print('丢失一个页面')
                    continue
            except:
                pass
            continue
        
print('成功！')   